# Multi-label prediction with Planet Amazon dataset

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *

## Get Data

In [ ]:
# ! pip install kaggle --upgrade

In [9]:
# ! mkdir -p .kaggle/
! mv kaggle.json .kaggle/

In [7]:
path = Path('data/planet')
os.makedirs(path, exist_ok=True)
path

PosixPath('data/planet')

In [12]:
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train-jpg.tar.7z -p {path}  
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train_v2.csv -p {path}  
# ! unzip {path}/train_v2.csv.zip -d {path}

100%|███████████████████████████████████████▉| 600M/600M [03:50<00:00, 2.93MB/s]
100%|████████████████████████████████████████| 600M/600M [03:50<00:00, 2.73MB/s]
100%|█████████████████████████████████████████| 159k/159k [00:00<00:00, 915kB/s]

Archive:  data/planet/train_v2.csv.zip
  inflating: data/planet/train_v2.csv  
   creating: data/planet/__MACOSX/
  inflating: data/planet/__MACOSX/._train_v2.csv  


In [13]:
# ! sudo apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package p7zip-full


In [14]:
# ! 7za -bd -y x {path}/train-jpg.tar.7z -o{path}
# ! tar -xf {path}/train-jpg.tar -C {path}


7-Zip (A) [64] 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18
p7zip Version 9.20 (locale=C.UTF-8,Utf16=on,HugeFiles=on,16 CPUs)

Processing archive: data/planet/train-jpg.tar.7z

Extracting  train-jpg.tar

Everything is Ok

Size:       665753600
Compressed: 629296207


In [16]:
path.ls()

[PosixPath('data/planet/train-jpg.tar'),
 PosixPath('data/planet/__MACOSX'),
 PosixPath('data/planet/train-jpg.tar.7z'),
 PosixPath('data/planet/train-jpg'),
 PosixPath('data/planet/train_v2.csv.zip'),
 PosixPath('data/planet/train_v2.csv')]

# Multiclass classification

In [17]:
df = pd.read_csv(path/'train_v2.csv')
df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


read in the dataset

In [18]:
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

In [19]:
np.random.seed()
data = (ImageFileList.from_folder(path)            
        .label_from_csv('train_v2.csv', sep=' ', folder='train-jpg', suffix='.jpg')  
        .random_split_by_pct(0.2)
        .datasets(ImageMultiDataset)  
        .transform(tfms, size=128)             
        .databunch()
        .normalize(imagenet_stats))